# Subtract average by row/column test

This notebook allows to test if subtracting by average from rows or columns of 2D plotted data works.

Start the plottr, and execute the code below. Below the code you will find the explanation on what to look at.

In [1]:
%matplotlib notebook
import numpy.random as rd
import matplotlib.pyplot as plt
from functools import partial
import numpy as np

from time import sleep, monotonic

import qcodes as qc
from qcodes import Station
from qcodes.dataset.experiment_container import (Experiment,
                                                 load_last_experiment,
                                                 new_experiment)

from qcodes.dataset.database import initialise_database
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.dataset.param_spec import ParamSpec
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.data_export import get_shaped_data_by_runid

# Plive plotting specific imports 
import plottr
from plottr.qcodes_dataset import QcodesDatasetSubscriber as LivePlotSubscriber 

In [2]:
initialise_database()
new_experiment(name='tutorial_exp', sample_name="no sample")

tutorial_exp#no sample#25@C:\Users\a-miasta/experiments.db
----------------------------------------------------------

In [3]:
# preparatory mocking of physical setup

dac = DummyInstrument('dac', gates=['x', 'y'])
dmm = DummyInstrument('dmm', gates=['model_row_value', 'model_column_value'])

station = qc.Station(dmm, dac)

In [4]:
meas = Measurement()
meas.register_parameter(dac.x)  # register the first independent parameter
meas.register_parameter(dac.y)  # register the second independent parameter
meas.register_parameter(dmm.model_row_value, setpoints=(dac.x, dac.y))  # now register the dependent one
meas.register_parameter(dmm.model_column_value, setpoints=(dac.x, dac.y))  # now register the dependent one

In [5]:
# add sample models with depend only on x or y

def model_with_rows():
    while True:
        (x, y) = yield
        yield -5 + y

def model_with_columns():
    while True:
        (x, y) = yield
        yield 5 + x

In [6]:
# and finally wire up the models to dmm channels

model_row = model_with_rows()
next(model_row)

model_column = model_with_columns()
next(model_column)

def measure_model_row(dac):
    val = model_row.send((dac.x.get(), dac.y.get()))
    next(model_row)
    return val

def measure_model_column(dac):
    val = model_column.send((dac.x.get(), dac.y.get()))
    next(model_column)
    return val

dmm.model_row_value.get = partial(measure_model_row, dac)
dmm.model_column_value.get = partial(measure_model_column, dac)

In [7]:
# run a 2D sweep

with meas.run() as datasaver:
    
    datasaver.dataset.subscribe(
        LivePlotSubscriber(datasaver.dataset),
        state=[], min_wait=0, min_count=1
    )

    for x_val in np.linspace(-1, 1, 80):
        for y_val in np.linspace(-1, 1, 100):
            dac.x(x_val)
            dac.y(y_val)
            
            row_val = dmm.model_row_value.get()
            column_val = dmm.model_column_value.get()
            
            datasaver.add_result((dac.x, x_val),
                                 (dac.y, y_val),
                                 (dmm.model_row_value, row_val),
                                 (dmm.model_column_value, column_val))
            
    dataid = datasaver.run_id

Starting experimental run with id: 62


## How to check that everything works?

The plottr window will plot the dataset with two measured parameters, each is a colormap of 2D data. Horizontal axis should be `dac_x`, vertical axis should be `dac_y`. One of the datasets, `dmm_model_column_value`, contains data which values grow along horizontal (x) axis, and in the other dataset, `dmm_model_row_value`, the values grow along vertical (y) axis. Click on each of them to explore the data.

### Test 0:

The radio button `None` should be clicked now in the `Subtract average` box. This means that subtracting average is not performed, hence the plotted data is not modified.

### Test 1:

* Select `dmm_model_column_value`. Values span from `4.0` to `6.0` and grow along the horizontal `dac_x` axis.
* Press `From each row` button in the `Subtract average` box. The data will look similar but now the values will span between `-1.0` and `1.0`.
* Press `From each column` button, and the data will look corrupted, with vertical stripes.

### Test 2:

* Select `dmm_model_row_value`. Values span from `-6.0` to `-4.0` and grow along the vertical `dac_y` axis.
* Press `From each column` button in the `Subtract average` box. The data will look similar but now the values will span between `-1.0` and `1.0`.
* Press `From each row` button, and the data will look corrupted, with horizontal stripes.